# textgrid demo (pytomofilt submodule)

This notebook demonstrates the `pytomofilt.textgrid` submodule (formerly `mantletext`).

It generates a regular lon/lat grid whose `z` values encode text in Robinson projection space.

Notes:
- If `font_path=None`, the default DejaVu font may be downloaded on first use and cached.
- To force offline behavior, set `PYTOMOFILT_TEXTGRID_DISABLE_FONT_DOWNLOAD=1` or pass `font_path=...`.


In [ ]:
from dataclasses import asdict

import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

from pytomofilt.textgrid import TextGridConfig, generate_text_grid, save_grid_to_xyz_file


In [ ]:
config = TextGridConfig(
    text="HELLO",
    center_lon=0.0,
    center_lat=0.0,
    size_deg=60.0,
    central_meridian=0.0,
    grid_spacing_deg=1.0,
    font_size=160,
)

df = generate_text_grid(config)
df.head()

In [ ]:
# Reshape to 2D for plotting
lons = np.sort(df["lon"].unique())
lats = np.sort(df["lat"].unique())

# df is produced by meshgrid ravel; rebuild in that same order
z = df["z"].to_numpy().reshape(len(lats), len(lons))

crs_plot = ccrs.Robinson(central_longitude=config.central_meridian)
crs_data = ccrs.PlateCarree()

fig = plt.figure(figsize=(11, 5))
ax = plt.axes(projection=crs_plot)
ax.set_global()
ax.coastlines(linewidth=0.6)
pc = ax.pcolormesh(lons, lats, z, transform=crs_data, cmap="Greys", shading="auto")
ax.set_title("textgrid mask (Robinson)")
plt.colorbar(pc, ax=ax, shrink=0.75, pad=0.03, label="z")
plt.show()

In [ ]:
# Save as an xyz grid (git-ignored output folder)
out_path = "output/textgrid_demo/hello.xyz"
save_grid_to_xyz_file(df, out_path, config_dict=asdict(config))
out_path